### Install Libraries

In [1]:
#use AIGeneratedText virtualenv for this project
#status - COde is not working , it has keras_nlp package issue 

In [1]:
# !pip install --upgrade keras
# !pip install --upgrade tensorflow
# !pip install "C:\Users\maste\Documents\Personal\GitHub\AIML_Projects\AIGenerated_TextDetection\external_packages\wheelhouse\keras_core-0.1.7-py3-none-any.whl"
# !pip install "C:\Users\maste\Documents\Personal\GitHub\AIML_Projects\AIGenerated_TextDetection\external_packages\wheelhouse\keras_nlp-0.6.4-py3-none-any.whl"
#!pip install "C:\Users\maste\Documents\Personal\GitHub\AIML_Projects\AIGenerated_TextDetection\external_packages\wheelhouse\keras-2.15.0-py3-none-any.whl"

# !pip install torch torchvision torchaudio
# !pip install pandas


In [2]:
#!pip install -q keras-core --upgrade
#!pip install keras
#!pip install tensorflow==2.13.0

#!pip install --upgrade keras-nlp

#!pip install --upgrade keras-nlp-nightly
#!pip install --upgrade keras



In [3]:
# !pip install keras==2.13.1
# !pip show keras


In [4]:
!pip show tensorflow

Name: tensorflow
Version: 2.10.1
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: c:\users\maste\anaconda3\envs\aigeneratedtext\lib\site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, keras-preprocessing, libclang, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: tensorflow-text


In [5]:
# !pip install keras-nlp==0.6.2

In [6]:
!pip show keras-nlp

Name: keras-nlp
Version: 0.7.0
Summary: Industry-strength Natural Language Processing extensions for Keras.
Home-page: https://github.com/keras-team/keras-nlp
Author: Keras team
Author-email: keras-nlp@google.com
License: Apache License 2.0
Location: c:\users\maste\anaconda3\envs\aigeneratedtext\lib\site-packages
Requires: absl-py, dm-tree, kagglehub, keras-core, numpy, packaging, regex, rich, tensorflow-text
Required-by: 


In [12]:
import os
os.environ["KERAS_BACKEND"]="torch"
import keras_core as core
import keras_nlp
import keras_core.backend as k
import tensorflow as tf

ImportError: cannot import name 'CachedMultiHeadAttention' from partially initialized module 'keras_nlp.src.layers.modeling.cached_multi_head_attention' (most likely due to a circular import) (C:\Users\maste\anaconda3\envs\AIGeneratedText\lib\site-packages\keras_nlp\src\layers\modeling\cached_multi_head_attention.py)

In [6]:


import torch

import numpy as np
import pandas as pd
import gc


In [13]:
print(keras.__version__)
#print(keras_nlp.__version__)
print(tf.__version__)

NameError: name 'keras' is not defined

In [32]:
print(keras.__version__)
#print(keras_nlp.__version__)
print(tf.__version__)

0.1.7
2.10.1


In [ ]:
!pip uninstall keras

### Config Setup

In [51]:
class confg:
    device='GPU'
    verbose=0
    seed =2023
    batch_size=5
    drop_remainder=True
    sequence_length=200
    class_name=['real','fake']
    num_classes=2
    pretrained_model='keras_trained'


keras.utils.set_random_seed(confg.seed)
    

In [6]:
def get_device():
    "detect and intialize GPU/TPU automatically"
    try:
        #connect to TPU
        tpu=tf.distribute.cluster_resolver.TPUClusterResolver.connect()
        strategy=tf.distribute.TPUStrategy(tpu)
        print(f"running on TPU {tpu.master()}")
        print("num of TPU's:",strategy.num_replicas_in_sync)
        device=config.device
    except:
        #connect GPU if TPU's not available
        gpus=tf.config.list_logical_devices('GPU')
        ngpus=len(gpus)
        if ngpus:
            print(f"running on gpu-no of GPU {ngpus}")
            #single GPU or MultiGPU
            strategy=tf.distribute.MirroredStrategy(gpus)
            device='GPU'
        else:
            #if no GPU's are available , use CPU
            print(f"running on CPU")
            strategy=tf.distribute.get_strategy()
            device='CPU'



        return strategy,device



strategy,confg.device=get_device()
confg.replicas=strategy.num_replicas_in_sync




      
        
        
        
        
    
    

running on CPU


### Test data

In [8]:
df_test=pd.read_csv("train_drcat_04.csv")
df_test

,essay_id,text,label
0,E897534557AF,"In recent years, technology has had a profoun...",1
1,DFBA34FFE11D,Should students participate in an extracurricu...,0
2,af37ecf5,The electoral college is a symbol of mockery a...,0
3,5EC2696BAD78,This is why I think the principle should allow...,0
4,llama_70b_v1843,I strongly believe that meditation and mindful...,1
...,...,...,...
44201,F7341069C4A4,"""Oh man I didn't make the soccer team!"", yelle...",0
44202,AFE6E553DAC2,I believe that using this technology could be ...,0
44203,falcon_180b_v1_600,The Face on Mars is a fascinating phenomenon t...,1
44204,A5F84C104693,Texting & Driving\n\nUsing your phone while dr...,0


### Preprocessing


In [14]:
vocab_path="vocab.spm/vocab.spm"   #vocublary file
tokenizer=keras_nlp.models.DebertaV3Tokenizer(vocab_path)  #tokenizer to convert test words into tokens
preprocessor=keras_nlp.models.DebertaV3Preprocessor(tokenizer,sequence_length=confg.sequence_length)


In [21]:
#converting only one value to check shape of tokens
outs=preprocessor(df_test['text'].iloc[0])
outs
for id,val in outs.items():
    print("shape of ",id,"is :",val.shape)

shape of  token_ids is : torch.Size([200])
shape of  padding_mask is : torch.Size([200])


In [22]:
#converting each and every text
def preprocessing_text(text,label=None):
    processedText=preprocessor(text)
    return (processedText,label) if label is not None else processedText

In [32]:
def build_dataset(texts,labels=None,batch_size=16,
                  cache=False,drop_remainder=True,
                  repeat=False,shuffle=1024):

    #to understand more about tf.data--> https://www.tensorflow.org/guide/data#basic_mechanics
    AUTO=tf.data.AUTOTUNE
    slices=(texts,) if labels is None else(texts,labels) #create slices
    dataset=tf.data.Dataset.from_tensor_slices(slices)
    dataset=dataset.cache() #cache dataset if enabled (enable cache only for CPU memort training)
    datset=dataset.map(preprocessing_text,num_parallel_calls=AUTO)
    dataset.repeat() if repeat else dataset
    opt=tf.data.Options()
    if shuffle:
        dataset=dataset.shuffle(shuffle,seed=confg.seed) #shuffle  dataset if enabled
        opt.experimental_deterministic=False

    dataset=dataset.batch(batch_size,drop_remainder=drop_remainder) #batching stacks n consecutive elements of a dataset into a single element.
    #Use the drop_remainder argument to ignore that last batch
    dataset=dataset.prefetch(AUTO)  #prefetch the next batch
    return dataset #return the build dataset
    
    

In [33]:
#generating dataset from test df
def get_texts_dataset(df_test):
    test_texts=df_test['text'].tolist()

    #build testation dataset
    test_dataset=build_dataset(test_texts,labels=None,batch_size=min(confg.batch_size*confg.replicas,len(df_test)),
                               cache=False,repeat=False,shuffle=False,drop_remainder=True)
    return test_dataset
    

## Prediction

In [44]:
!pip install tqdm

  Using cached tqdm-4.66.1-py3-none-any.whl.metadata (57 kB)
Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)


In [79]:
#intialize an array to store predictions for each fold
from tqdm.notebook import tqdm
import os
fold_preds=np.zeros(shape=(len(df_test),),dtype='float32')
#iterated through each trained fold
for fld in tqdm(os.listdir(confg.pretrained_model)):
    #load the pre trained model saved in local directory
    fld=os.path.join(r"C:\Users\maste\Documents\Personal\GitHub\AIML_Projects\AIGenerated_TextDetection\keras_trained",fld)
    #fld=r"‪C:/Users/maste/Documents/Personal/GitHub/AIML_Projects/AIGenerated_TextDetection/keras_trained/fold0_.keras.keras"
    print(fld)
    model= keras.models.load_model(fld,compile=False,safe_mode=True,)
    #call function to get test dataset
    #test_dataset=get_texts_dataset(df_test)

     

  0%|          | 0/3 [00:00<?, ?it/s]

C:\Users\maste\Documents\Personal\GitHub\AIML_Projects\AIGenerated_TextDetection\keras_trained\fold0_.keras


C:\Users\maste\anaconda3\envs\AIGeneratedText\lib\site-packages\keras_core\src\trainers\trainer.py:66: UserWarning: `jit_compile` is not yet enabled for the PyTorch backend. Proceeding with `jit_compile=False`.
  warnings.warn(
C:\Users\maste\anaconda3\envs\AIGeneratedText\lib\site-packages\keras_core\src\saving\serialization_lib.py:686: UserWarning: `compile()` was not called as part of model loading because the model's `compile()` method is custom. All subclassed Models that have `compile()` overridden should also override `get_compile_config()` and `compile_from_config(config)`. Alternatively, you can call `compile()` manually after loading.
  instance.compile_from_config(compile_config)


ValueError: Layer 'token_embedding' expected 1 variables, but received 0 variables during loading. Expected: ['embeddings']

In [59]:
for fld in tqdm(os.listdir(confg.pretrained_model)):
    print(os.path.join(confg.pretrained_model,fld))
    

  0%|          | 0/3 [00:00<?, ?it/s]

keras_trained\fold0_.keras
keras_trained\fold1_.keras
keras_trained\fold2_.keras


In [73]:
for file in os.listdir(r"C:\Users\maste\Documents\Personal\GitHub\AIML_Projects\AIGenerated_TextDetection\keras_trained"):
    print(file)


fold0_.keras
fold1_.keras
fold2_.keras
